## NLP with sklearn and spacy

In [180]:
import json
from pprint import pprint
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
import string
punctuations = string.punctuation

In [15]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Electronics_5.json.gz')

In [112]:
df.shape

(1689188, 9)

In [114]:
train_set = df[0:500000]
train_set.shape

(500000, 9)

In [169]:
test_set = df[500001:500501]
test_set.shape

(500, 9)

In [171]:
def good_or_bad_func(rating):
    if rating > 3:
        return 'pos'
    else:
        return 'neg'
train_set['good_or_bad'] = train_set.overall.map(good_or_bad_func)
test_set['good_or_bad'] = test_set.overall.map(good_or_bad_func)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [172]:
test_set

,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,overall,reviewTime,summary,good_or_bad
500001,A2L5R1F9WALW7W,B001NPDA44,"K. Bunker ""K. Bunker""","[286, 294]",1235520000,Does what it's supposed to do and looks fine t...,5.0,"02 25, 2009",Perfectly good for what it is,pos
500002,A3NEMJ243HIIIK,B001NPDA44,Kevin Hutchinson,"[2, 2]",1257033600,"As a vegetarian, I was looking for a non-leath...",5.0,"11 1, 2009",Just perfect if you prefer to handle the Kindl...,pos
500003,A1VASOVPUGUYEJ,B001NPDA44,K. Holmes,"[1, 1]",1249516800,"Before I even ordered my Kindle, I read as muc...",5.0,"08 6, 2009",Just what I wanted and needed for my K2,pos
500004,A12F8OG8CKZU9L,B001NPDA44,kjvp,"[0, 0]",1280448000,Bought this together with the Clear MCover Har...,5.0,"07 30, 2010",Great product,pos
500005,A36WET2V5EEN6U,B001NPDA44,K&K,"[1, 1]",1287014400,If you own a Kindle 3 with the Lighted or non ...,5.0,"10 14, 2010",Fit's Kindle 3 in Light Book cover,pos
500006,A36B7TZNERS5IW,B001NPDA44,Krykie,"[4, 4]",1243036800,The sleeve isn't bulky and doesn't take up too...,4.0,"05 23, 2009","Yes, its worth it",pos
500007,A37V0OXJHF658O,B001NPDA44,Kurtis Simmons,"[0, 0]",1316995200,"This case is perfect for your Kindle. Sleek, ...",4.0,"09 26, 2011",What an excellent product!,pos
500008,A1PLXA4HC927ZF,B001NPDA44,"K. Via ""Kevin""","[47, 50]",1235433600,The Belkin sleeve does what its supposed to do...,5.0,"02 24, 2009",Does what its supposed to do,pos
500009,AL7ZQN0HVE2SE,B001NPDA44,Linda from Louisville,"[1, 1]",1240444800,"Like others have commented, I wasn't intereste...",5.0,"04 23, 2009",Perfect For My Purposes,pos
500010,ADIGT993RO2K7,B001NPDA44,Livingston,"[1, 1]",1308268800,"Now, before you go buy this and think that it ...",5.0,"06 17, 2011",Great for Your Kindle 3,pos


In [18]:
import spacy

In [19]:
nlp = spacy.en.English()

In [121]:
reviews = nlp(train_set.iloc[1].reviewText.decode('utf-8'))

In [122]:
dir(reviews)

['__bytes__',
 '__class__',
 '__delattr__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getitem__',
 '__hash__',
 '__init__',
 '__iter__',
 '__len__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_py_tokens',
 '_realloc',
 '_vector',
 '_vector_norm',
 'count_by',
 'doc',
 'ents',
 'from_array',
 'from_bytes',
 'has_vector',
 'is_parsed',
 'is_tagged',
 'mem',
 'merge',
 'noun_chunks',
 'noun_chunks_iterator',
 'read_bytes',
 'sentiment',
 'sents',
 'similarity',
 'string',
 'tensor',
 'text',
 'text_with_ws',
 'to_array',
 'to_bytes',
 'user_data',
 'user_hooks',
 'user_span_hooks',
 'user_token_hooks',
 'vector',
 'vector_norm',
 'vocab']

In [123]:
#Me: hey there's an object for sentiment already! Yayyyyy!
#Spacy: In your dreams son!
reviews.sentiment

0.0

In [38]:
#first word
reviews[0]

I

In [40]:
#last word
reviews[-2]

table

In [41]:
list(reviews.sents)

[I bought this remote to make it easier for my husband to use our television system.  ,
 We don't have a lot connected; only a new TV, blu-ray, AV receiver, and old VCR, but that took four, sometimes five, remotes depending on what one wanted to do.  ,
 It got confusing.,
 Although I'm comfortable sitting at a computer (this is how one must program the Harmony One), I felt it was rather d-u-m to have to keep getting up and going out to the viewing room to check to make sure the programming was correct.  ,
 Especially if one has to do this multiple times be it errors or changes/additions.,
 I only made one error on the initial set-up and got it corrected with no outside help.  ,
 The software could be laid out a bit easier; I was stumbling at first and the paperwork the remote comes with isn't extremely helpful.,
 Once set up, there's a bit of a learning curve on it as each piece of equipment has its own set of buttons and one has to find them on other screens.  ,
 Sometimes, depending 

In [126]:
#get all tags
all_tags = {w.pos: w.pos_ for w in reviews}
all_tags

{82: u'ADJ',
 83: u'ADP',
 84: u'ADV',
 87: u'CCONJ',
 88: u'DET',
 89: u'INTJ',
 90: u'NOUN',
 91: u'NUM',
 92: u'PART',
 93: u'PRON',
 94: u'PROPN',
 95: u'PUNCT',
 97: u'SYM',
 98: u'VERB',
 101: u'SPACE'}

In [127]:
#first sentence
for word in list(reviews.sents)[0]:  
    print word, word.tag_

I PRP
'm VBP
a DT
professional JJ
OTR NNP
truck NN
driver NN
, ,
and CC
I PRP
bought VBD
a DT
TND NNP
700 CD
at IN
a DT
truck NN
stop NN
hoping VBG
to TO
make VB
my PRP$
life NN
easier JJR
. .
  SP


In [128]:
#define some parameters  
noisy_pos_tags = ['PROP']
min_token_length = 2

#Function to check if the token is a noise or not  
def isNoise(token):     
    is_noise = False
    if token.pos_ in noisy_pos_tags:
        is_noise = True 
    elif token.is_stop == True:
        is_noise = True
    elif len(token.string) <= min_token_length:
        is_noise = True
    return is_noise 
def cleanup(token, lower = True):
    if lower:
       token = token.lower()
    return token.strip()

# top unigrams used in the reviews 
from collections import Counter
cleaned_list = [cleanup(word.string) for word in reviews if not isNoise(word)]
Counter(cleaned_list) .most_common(5)

[(u'truck', 7), (u'700', 5), (u'tnd', 5), (u'pickup', 4), (u'route', 4)]

In [129]:
labels = set([w.label_ for w in reviews.ents]) 
for label in labels: 
    entities = [cleanup(e.string, lower=False) for e in reviews.ents if label==e.label_] 
    entities = list(set(entities)) 
    print label,entities

MONEY [u'80,000#']
TIME [u'almost 5 minutes']
PERSON [u'Rand McNally']
CARDINAL [u'I-71', u'700', u'$30', u'53', u'SD']
GPE [u'Cincinnati', u'PO']
DATE [u'every six months or so', u'weekly']
ORG [u'Google', u'Phoenix AZ', u'GPS (Sprint', u'TND', u'the Cincinnati OH', u'I-75 I', u'OTR']
QUANTITY [u'thru 23 miles', u'about 15 miles']


In [140]:
remote = [sent for sent in reviews.sents if 'truck' in sent.string.lower()]
# create dependency tree
sentence = remote[2]
print remote
for word in sentence:
    print word, ': ', str(list(word.children))

[I'm a professional OTR truck driver, and I bought a TND 700 at a truck stop hoping to make my life easier.  , I live in my truck and at truck stops.  , If you need to make a patch available on an SD card then you should send the SD cards to the truck stops where the devices are sold.  , I was parked at a truck stop in the Cincinnati OH area.  , Very exciting, especially since every time I refused to turn down a street posted "No Trucks" the TND 700 took almost 5 minutes to figure a re-route, and it happened multiple times on that short trip., I turned it off and the next time I passed a truck stop of the same chain I purchased it at I returned it and got my money back.]
If :  []
you :  []
need :  [If, you, make]
to :  []
make :  [to, available]
a :  []
patch :  [a]
available :  [patch, on]
on :  [card]
an :  []
SD :  []
card :  [an, SD]
then :  []
you :  []
should :  []
send :  [need, then, you, should, cards, to, stops, .]
the :  []
SD :  []
cards :  [the, SD]
to :  [truck]
the :  []

In [142]:
def pos_words (sentence, token, ptag):
    sentences = [sent for sent in reviews.sents if token in sent.string]     
    pwrds = []
    for sent in sentences:
        for word in sent:
            pwrds.extend([child.string.strip() for child in word.children if child.pos_ == ptag] )
    return Counter(pwrds).most_common(10)

pos_words(reviews, 'truck', 'ADJ')

[(u'my', 3),
 (u'available', 1),
 (u'same', 1),
 (u'next', 1),
 (u'professional', 1),
 (u'easier', 1)]

## Sentiment Analyser

In [143]:
parser = spacy.en.English()
#Custom transformer using spaCy 
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

# Basic utility function to clean the text 
def clean_text(text):     
    return text.strip().lower()

In [144]:
#Create spacy tokenizer that parses a sentence and generates tokens
#these can also be replaced by word vectors 
def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    tokens = [tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_ for tok in tokens]
    tokens = [tok for tok in tokens if (tok not in stopwords and tok not in punctuations)]
    return tokens

#create vectorizer object to generate feature vectors, we will use custom spacy’s tokenizer
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
classifier = LinearSVC()

In [145]:
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])

In [160]:
"""
started with the whole review but ppl had too much time to write a whole book for every item they purchased, 
hence had to use the summary text which was too short but what the hell!
"""
train = zip(train_set.summary.tolist(),train_set.good_or_bad.tolist())

In [161]:
pipe.fit([x[0] for x in train], [x[1] for x in train]) 

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000000006B18E748>), ('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
   ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [173]:
test = zip(test_set.summary.tolist(),test_set.good_or_bad.tolist())

In [163]:
test =   [('the beer was good.', 'pos'),     
         ('I do not enjoy my job', 'neg'),
         ("I ain't feelin dandy today.", 'neg'),
         ("I feel amazing!", 'pos'),
         ('Gary is a good friend of mine.', 'pos'),
         ("I can't believe I'm doing this.", 'neg')]

In [174]:
pred_data = pipe.predict([x[0] for x in test])

In [175]:
for (sample, pred) in zip(test, pred_data):
    print sample, pred

print "Accuracy:", accuracy_score([x[1] for x in test], pred_data)

('Perfectly good for what it is', 'pos') pos
('Just perfect if you prefer to handle the Kindle by itself', 'pos') pos
('Just what I wanted and needed for my K2', 'pos') pos
('Great product', 'pos') pos
("Fit's Kindle 3 in Light Book cover", 'pos') pos
('Yes, its worth it', 'pos') pos
('What an excellent product!', 'pos') pos
('Does what its supposed to do', 'pos') pos
('Perfect For My Purposes', 'pos') pos
('Great for Your Kindle 3', 'pos') pos
('great belkin case', 'pos') pos
('Simple, effective protection', 'pos') pos
('Actually fit in my Verso clip on light in with my 6" Kindle', 'pos') pos
('Just fine!', 'pos') pos
('Stylish protective case', 'pos') pos
('Well-made, no frills, does the job', 'pos') pos
('A bit too bulky for my smaller kindle but perfect for my USB DVD-Writer', 'pos') pos
('A Snug and Secure Home for My Kindle...', 'pos') pos
('very good cover', 'pos') pos
('Nice little sleeve', 'pos') pos
('Truth in advertising', 'pos') neg
('Great for Kindle 3 too', 'pos') pos
("I

In [176]:
# trying out the same thing with the review texts
train_set_full_reviews = df[0:5000]
train_set_full_reviews['good_or_bad'] = train_set_full_reviews.overall.map(good_or_bad_func)
trainX = zip(train_set_full_reviews.reviewText.tolist(),train_set_full_reviews.good_or_bad.tolist())
pipe.fit([x[0] for x in trainX], [x[1] for x in trainX])

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


('the beer was good.', 'pos') pos
('I do not enjoy my job', 'neg') pos
("I ain't feelin dandy today.", 'neg') pos
('I feel amazing!', 'pos') pos
('Gary is a good friend of mine.', 'pos') pos
("I can't believe I'm doing this.", 'neg') pos
Accuracy: 0.5


In [182]:

test =   [('the beer was good.', 'pos'),     
         ('I do not enjoy my job', 'neg'),
         ("this NLP thing is cool.", 'cool'),
         ("the pizza I just had was nice", 'pos'),
         ('Bao is a good friend of mine', 'pos'),
         ("I can't believe I'm doing this.", 'neg')]
pred_data = pipe.predict([x[0] for x in test])
for (sample, pred) in zip(test, pred_data):
    print sample, pred

print "Accuracy:", accuracy_score([x[1] for x in test], pred_data)

('the beer was good.', 'pos') pos
('I do not enjoy my job', 'neg') pos
('this NLP thing is cool.', 'cool') pos
('the pizza I just had was nice', 'pos') pos
('Bao is a good friend of mine', 'pos') pos
("I can't believe I'm doing this.", 'neg') pos
Accuracy: 0.5


In [177]:
test = zip(test_set.summary.tolist(),test_set.good_or_bad.tolist())
pred_data = pipe.predict([x[0] for x in test])
for (sample, pred) in zip(test, pred_data):
    print sample, pred

print "Accuracy:", accuracy_score([x[1] for x in test], pred_data)

('Perfectly good for what it is', 'pos') pos
('Just perfect if you prefer to handle the Kindle by itself', 'pos') pos
('Just what I wanted and needed for my K2', 'pos') pos
('Great product', 'pos') pos
("Fit's Kindle 3 in Light Book cover", 'pos') pos
('Yes, its worth it', 'pos') pos
('What an excellent product!', 'pos') pos
('Does what its supposed to do', 'pos') pos
('Perfect For My Purposes', 'pos') pos
('Great for Your Kindle 3', 'pos') pos
('great belkin case', 'pos') pos
('Simple, effective protection', 'pos') pos
('Actually fit in my Verso clip on light in with my 6" Kindle', 'pos') pos
('Just fine!', 'pos') pos
('Stylish protective case', 'pos') pos
('Well-made, no frills, does the job', 'pos') pos
('A bit too bulky for my smaller kindle but perfect for my USB DVD-Writer', 'pos') pos
('A Snug and Secure Home for My Kindle...', 'pos') pos
('very good cover', 'pos') pos
('Nice little sleeve', 'pos') pos
('Truth in advertising', 'pos') pos
('Great for Kindle 3 too', 'pos') pos
("I